In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pyfolio
import json
import numpy as np
import pandas as pd
from params import *

from tqdm import tqdm
from itertools import combinations
from utils.read2df import read2df
from utils.cointncorr import CointnCorr

In [ ]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t um -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t um -skip-daily 1

In [3]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs, marketType='futures/um')

# have a preview
dfs[0]

,time,open,high,low,close,volume,tic,itvl,datetime
0,1610434859999,36060.5,36060.5,36060.5,36060.5,0.469,BTCBUSD,1m,2021-01-12 07:00:59.999
1,1610434859999,36026.67,36150.0,36001.5,36112.93,287.146,BTCUSDT,1m,2021-01-12 07:00:59.999
2,1610434919999,36042.0,36042.0,36042.0,36042.0,0.001,BTCBUSD,1m,2021-01-12 07:01:59.999
3,1610434919999,36112.95,36112.95,35900.75,35900.75,229.559,BTCUSDT,1m,2021-01-12 07:01:59.999
4,1610434979999,36042.0,36042.0,35856.5,35856.5,0.906,BTCBUSD,1m,2021-01-12 07:02:59.999
...,...,...,...,...,...,...,...,...,...
2945395,1698796679999,34660.40,34677,34660.30,34677,69.881,BTCUSDT,1m,2023-10-31 23:57:46.880
2945396,1698796739999,34669.8,34669.9,34669.8,34669.9,0.687,BTCBUSD,1m,2023-10-31 23:59:57.952
2945397,1698796739999,34677,34677,34670.80,34671.70,54.919,BTCUSDT,1m,2023-10-31 23:59:57.952
2945398,1698796799999,34669.9,34669.9,34653.8,34653.8,4.307,BTCBUSD,1m,2023-10-31 23:59:57.952


In [7]:
dfs[0].dtypes

time                object
open                object
high                object
low                 object
close               object
volume              object
tic                 object
itvl                object
datetime    datetime64[ns]
dtype: object

In [4]:
'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []']'
for i in range(len(dfs)):
    trains.append(dfs[i][(dfs[i]['datetime'] > start_date) & (dfs[i]['datetime'] < trade_date)].reset_index(drop=True))
    tests.append(dfs[i][dfs[i]['datetime'] >= trade_date].reset_index(drop=True))

In [5]:
'''
Find the best pair
'''

import pickle

tables = CointnCorr(trains, freqs).tabulate()
cointncorrtxt = f"result/gridsearch/cointncorr.txt"

os.remove(f"{cointncorrtxt}") if os.path.exists(f"{cointncorrtxt}") else None

with open(cointncorrtxt, "w+") as f:
    for k, v in tables.items():
        f.write(f"{k}\n")
        f.write(f"{v}\n\n")
f.close()

best_value = 0
for key in tables.keys():
    for freq in freqs:
        rel = tables[key].at['coint', freq] + tables[key].at['corr', freq]
        if rel > best_value:
            best_value = rel
            best_pair = key
            best_freq = freq

print("===========================================")
print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
print("===========================================")

cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)

c:\Users\hyan212\backtrader-crypto-rl\utils\cointncorr.py:68: CollinearityWarning: y0 and y1 are (almost) perfectly colinear.Cointegration test is not reliable in this case.
  _, pvalue, _ = coint(first_ele, second_ele)


Best trading pairs shall be: BTCBUSD_BTCUSDT under 1m interval
the coint is 93.29%
and the corr is 0.999
